In [17]:
from itertools import product
import pandas as pd
import numpy as np
import os,sys
from datetime import datetime,timedelta
from multiprocessing import Pool
import matplotlib.pyplot as plt
from scipy import stats

%matplotlib inline

os.chdir('/data/arpit.goel/33_GrocerySales/')

dtypes={'id':'category','store_nbr':'category','item_nbr':'category','unit_sales':np.float64,'onpromotion':np.bool}

means=pd.read_csv('05.Features/01.Item_Store_Means_2017-07-01_182.csv',dtype=dtypes)
items=pd.read_pickle('02.ParsedData/items.pkl')
stores=pd.read_pickle('02.ParsedData/stores.pkl')


In [26]:
data_rich_points=means[means['count_promo']>10]
data_rich_points=data_rich_points[data_rich_points['count_no_promo']>10]
data_rich_points['harmonic_mean']=data_rich_points[['count_promo','count_no_promo']].apply(stats.hmean,axis=1)
print (data_rich_points['harmonic_mean'].describe(percentiles=[0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.95,0.99]))

data_rich_points=data_rich_points[data_rich_points['harmonic_mean']>40]
data_rich_points['promo_pct']=data_rich_points['avg_promo']/data_rich_points['avg_no_promo']-1
#data_rich_points=data_rich_points[['item_nbr','store_nbr','promo_pct','avg_promo','avg_no_promo']]
data_rich_points=pd.merge(data_rich_points,items,on='item_nbr')
data_rich_points=pd.merge(data_rich_points,stores,on='store_nbr')
data_rich_points.to_csv('05.Features/02.PromoDiff_2017-07-01_182.csv',index=False)

count    65425.000000
mean        44.064915
std         16.667192
min         11.000000
1%          16.611111
5%          20.622222
10%         23.790850
25%         31.836735
50%         42.212766
75%         54.393162
90%         68.794702
95%         76.563882
99%         85.953488
max         90.824176
Name: harmonic_mean, dtype: float64


In [25]:
data_rich_points.head()

,store_nbr,item_nbr,avg,avg_no_promo,avg_promo,avg_mon,avg_tue,avg_wed,avg_thur,avg_fri,...,count_sun_no_promo,count_mon_promo,count_tue_promo,count_wed_promo,count_thur_promo,count_fri_promo,count_sat_promo,count_sun_promo,harmonic_mean,promo_pct
8,1,1009997,1.188640,1.163328,1.236000,1.181467,1.182333,1.212800,1.275857,1.150917,...,4.0,5.0,4.0,7.0,6.0,2.0,3.0,4.0,40.404494,0.062469
15,1,1014864,1.451297,1.391475,1.588614,1.497050,1.479000,1.579640,1.363944,1.377391,...,14.0,6.0,7.0,8.0,5.0,6.0,7.0,5.0,61.296552,0.141676
31,1,1036689,2.044028,2.007423,2.272440,1.905308,2.410538,2.115385,1.990654,2.096346,...,22.0,3.0,3.0,4.0,3.0,4.0,5.0,3.0,43.093923,0.132018
47,1,1039087,1.212531,0.803469,1.417062,1.336786,1.195000,1.208333,1.053143,1.238687,...,1.0,9.0,11.0,9.0,10.0,9.0,10.0,6.0,42.666667,0.763681
56,1,1047679,3.283453,3.062659,3.751690,3.279654,3.386346,3.480346,3.480500,3.367500,...,17.0,8.0,8.0,9.0,8.0,8.0,9.0,8.0,78.828729,0.224978


In [29]:
train=pd.read_pickle('02.ParsedData/train_2016.pkl')
train['weekday']=(train['date']-4)%7

In [30]:
train['weekday_1']=(train['date']-4)%7
train['weekday_2']=train['date'].map(lambda x: datetime(1970,1,1)+timedelta(days=x))
train['weekday_3']=train['weekday_2'].dt.weekday


In [31]:
train[['weekday_1','weekday_2','weekday_3']].drop_duplicates().head(25)

,weekday_1,weekday_2,weekday_3
0,4,2016-01-01,4
1718,5,2016-01-02,5
102543,6,2016-01-03,6
202907,0,2016-01-04,0
299462,1,2016-01-05,1
393916,2,2016-01-06,2
487545,3,2016-01-07,3
577878,4,2016-01-08,4
669444,5,2016-01-09,5
769365,6,2016-01-10,6


In [36]:
test=pd.read_pickle('02.ParsedData/test.pkl')
test['date_1']=test['date'].map(lambda x: datetime(1970,1,1)+timedelta(days=x))


In [37]:
test[(test['item_nbr']=='1503844') & (test['store_nbr']=='44')]

,date,store_nbr,item_nbr,onpromotion,id,date_1
170428,17394,44,1503844,0,125667468,2017-08-16
381082,17395,44,1503844,2,125878122,2017-08-17
591736,17396,44,1503844,2,126088776,2017-08-18
802390,17397,44,1503844,2,126299430,2017-08-19
1013044,17398,44,1503844,2,126510084,2017-08-20
1223698,17399,44,1503844,2,126720738,2017-08-21
1434352,17400,44,1503844,2,126931392,2017-08-22
1645006,17401,44,1503844,0,127142046,2017-08-23
1855660,17402,44,1503844,2,127352700,2017-08-24
2066314,17403,44,1503844,2,127563354,2017-08-25


In [43]:
list_weekday=['mon','tue','wed','thur','fri','sat','sun']
promo_diff=[means['avg_%s_promo'%x]-means['avg_%s_no_promo'%x] for x in list_weekday]
pd.DataFrame(promo_diff).T.apply(lambda x: np.nanmean(x),axis=1)

/data/arpit.goel/anaconda/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: Mean of empty slice
  app.launch_new_instance()


0              NaN
1         0.529484
2              NaN
3              NaN
4         0.490055
5         0.526893
6              NaN
7              NaN
8         0.098005
9         0.320543
10       -0.128106
11        0.147501
12        0.357369
13             NaN
14        0.223547
15        0.195044
16             NaN
17             NaN
18             NaN
19        0.039675
20             NaN
21       -0.185389
22        0.715125
23        0.365906
24             NaN
25        0.254525
26        0.145771
27             NaN
28             NaN
29        0.082128
            ...   
165535   -0.359879
165536    0.065385
165537    0.220467
165538    0.059462
165539   -0.034854
165540         NaN
165541         NaN
165542   -0.115500
165543         NaN
165544         NaN
165545         NaN
165546         NaN
165547         NaN
165548    0.180977
165549         NaN
165550         NaN
165551    0.072772
165552         NaN
165553         NaN
165554         NaN
165555         NaN
165556    0.

In [44]:
means

,store_nbr,item_nbr,avg,avg_no_promo,avg_promo,avg_mon,avg_tue,avg_wed,avg_thur,avg_fri,...,count_fri_no_promo,count_sat_no_promo,count_sun_no_promo,count_mon_promo,count_tue_promo,count_wed_promo,count_thur_promo,count_fri_promo,count_sat_promo,count_sun_promo
0,1,1000866,0.764667,0.764667,NaN,0.774200,0.693000,0.831600,0.693000,0.760667,...,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1001305,1.028500,0.989943,1.507714,0.919917,1.078000,1.062000,1.116133,0.956313,...,14.0,14.0,8.0,NaN,2.0,1.0,NaN,2.0,1.0,1.0
2,1,1003679,1.208244,1.208244,NaN,1.259412,1.286579,1.263913,1.302682,1.155375,...,24.0,16.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1004545,0.693000,0.693000,NaN,0.693000,0.693000,0.693000,NaN,0.693000,...,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1004550,2.577417,2.520987,2.991238,2.074000,2.302500,2.538000,2.809423,3.049923,...,23.0,21.0,19.0,2.0,3.0,5.0,2.0,3.0,3.0,3.0
5,1,1004551,2.398186,2.374503,2.793700,1.924962,2.113077,2.616500,2.470808,2.816808,...,25.0,24.0,22.0,1.0,3.0,2.0,2.0,1.0,NaN,1.0
6,1,1009512,0.794500,0.794500,NaN,0.794500,0.693000,0.743750,0.693000,0.794500,...,8.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1,1009539,0.770800,0.770800,NaN,0.794500,0.693000,0.815111,0.963667,0.693000,...,5.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1,1009997,1.188640,1.163328,1.236000,1.181467,1.182333,1.212800,1.275857,1.150917,...,10.0,10.0,4.0,5.0,4.0,7.0,6.0,2.0,3.0,4.0
9,1,1009998,1.077761,1.014043,1.356905,1.128750,1.059353,1.182042,1.023500,1.084111,...,15.0,14.0,6.0,4.0,1.0,5.0,2.0,3.0,3.0,3.0
